Building a Chatbot with history

In [217]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

In [218]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B2940E780>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B2940EC40>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [219]:
from langchain_core.messages import HumanMessage
response=llm.invoke([HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai")])
response.content

"Hello Sachin,\n\nIt's great to meet you! \n\nPursuing a B.Tech in CSE from Thapar University is impressive, and it sounds like you're diving into a fascinating field with Generative AI. \n\nWhat aspects of Generative AI are you most interested in practicing? \n\nI'm always happy to chat about AI, whether it's about specific techniques, applications, or even the ethical considerations surrounding it. Feel free to ask me anything!\n"

In [220]:
from langchain_core.messages import AIMessage
llm.invoke(
    [
        HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ,
        AIMessage(content="Hi Sachin,\n\nIt's great to meet you!  It's impressive that you're already practicing Generative AI during your B.Tech in CSE at Thapar University.  That's a very forward-looking skillset to be developing.  What specific areas of Generative AI are you focusing on right now?  I'd be interested to hear more about your projects.") ,
        HumanMessage(content="What is my name?")
    ]
)


AIMessage(content="Your name is Sachin Goyal.  I remember! 😊 \n\nIs there anything specific you'd like to learn more about regarding Generative AI?  We could discuss:\n\n* **Specific models:**  Like GPT-3, DALL-E 2, Stable Diffusion, etc.\n* **Techniques:**  Such as diffusion models, transformer networks, GANs.\n* **Applications:**  Like text generation, image creation, music composition, code generation.\n* **Ethical considerations:**  Bias, fairness, misuse potential.\n* **Resources for learning:**  Books, courses, online communities. \n\nLet me know what piques your interest!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 152, 'total_tokens': 291, 'completion_time': 0.252727273, 'prompt_time': 0.006549447, 'queue_time': 0.24760658200000002, 'total_time': 0.25927672}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6594ffc8-f562-4102-ae87-269e45229dd1-0',

In [221]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #storing all the chat with their id to uniquely define and prevent chat mixing
def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:  #if new chat then store the chat going on
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [222]:
#declaring session id chat1 for this chat for a user
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke (
    [ HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ]
    ,config=config )
response

AIMessage(content='Hey Sachin,\n\nIt\'s awesome that you\'re diving into the world of Generative AI during your B.Tech! That\'s a super exciting field with tons of potential. \n\nSince you\'re already practicing, let\'s focus on boosting your learning even further. Here\'s a breakdown of suggestions:\n\n**1. Deepen Your Foundations:**\n\n* **Machine Learning:** Generative AI heavily relies on ML concepts.  Make sure you have a strong grasp of:\n    * Supervised vs. Unsupervised Learning\n    * Regression vs. Classification\n    * Common algorithms (Linear Regression, Decision Trees, SVMs, etc.)\n    * Evaluation metrics (Accuracy, Precision, Recall, F1-Score)\n* **Deep Learning:** Generative AI often uses deep neural networks. Familiarize yourself with:\n    * Convolutional Neural Networks (CNNs) for image generation\n    * Recurrent Neural Networks (RNNs) and Long Short-Term Memory (LSTM) networks for text generation\n    * Generative Adversarial Networks (GANs) - the cornerstone of m

In [223]:
#see this time it says i don't know about you name
#basically the session id is a new one so no previous chat available it will be stored for the first time

config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 13, 'total_tokens': 50, 'completion_time': 0.067272727, 'prompt_time': 0.001834105, 'queue_time': 0.25129953400000005, 'total_time': 0.069106832}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1d0c0c48-8e8c-4d8c-ae03-08de56114d0e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 37, 'total_tokens': 50})

In [224]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="ok tell me what is my name")],config=config
)
response
#see it remembers my name

AIMessage(content='Your name is Sachin Goyal. 😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 784, 'total_tokens': 797, 'completion_time': 0.023636364, 'prompt_time': 0.026718501, 'queue_time': 0.2492606, 'total_time': 0.050354865}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--be935fe9-866e-4247-a8ef-fbf4aae877ac-0', usage_metadata={'input_tokens': 784, 'output_tokens': 13, 'total_tokens': 797})

In [225]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="My name is John")],config=config)
response

AIMessage(content="Hi John! It's nice to meet you. 😊\n\nHow can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 62, 'total_tokens': 85, 'completion_time': 0.041818182, 'prompt_time': 0.003502289, 'queue_time': 0.25167511200000003, 'total_time': 0.045320471}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--af940eb7-76f0-4138-bba2-3f49204f499c-0', usage_metadata={'input_tokens': 62, 'output_tokens': 23, 'total_tokens': 85})

In [226]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response
#this time remembers the name

AIMessage(content='Your name is John, you told me earlier!  😄  \n\nIs there anything else I can help you with, John?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 97, 'total_tokens': 126, 'completion_time': 0.052727273, 'prompt_time': 0.004176181, 'queue_time': 0.247585548, 'total_time': 0.056903454}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--43bf4b27-81b9-4144-9e2e-3a4d8704453b-0', usage_metadata={'input_tokens': 97, 'output_tokens': 29, 'total_tokens': 126})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [227]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [228]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")]})

AIMessage(content="Hello Sachin Goyal,\n\nIt's nice to meet you! How can I help you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 28, 'total_tokens': 54, 'completion_time': 0.047272727, 'prompt_time': 0.002085065, 'queue_time': 0.246472886, 'total_time': 0.049357792}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--35a20333-7398-45a6-bb7d-b71078073a13-0', usage_metadata={'input_tokens': 28, 'output_tokens': 26, 'total_tokens': 54})

In [229]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [230]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="My name is Sachin Goyal")]},config=config
)
response

AIMessage(content="I know! You told me at the beginning.  \n\nIt's nice to meet you, Sachin. How can I help you further today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 821, 'total_tokens': 854, 'completion_time': 0.06, 'prompt_time': 0.028214673, 'queue_time': 0.250391636, 'total_time': 0.088214673}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--425f38b8-0667-4f8c-8536-b8c9d7a1abd9-0', usage_metadata={'input_tokens': 821, 'output_tokens': 33, 'total_tokens': 854})

In [231]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="can you tell me what is my age and my fav hobby")]},config=config
)
response

AIMessage(content='As an AI, I don\'t have access to personal information about you like your age or favorite hobby. \n\nTo tell me about yourself, you can simply say:\n\n* "I\'m 20 years old."\n* "My favorite hobby is playing basketball." \n\nI\'m here to learn more about you! 😄  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 874, 'total_tokens': 949, 'completion_time': 0.136363636, 'prompt_time': 0.029559817, 'queue_time': 0.250290624, 'total_time': 0.165923453}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--50f358d9-0f7e-41b1-9b3e-736f8633240d-0', usage_metadata={'input_tokens': 874, 'output_tokens': 75, 'total_tokens': 949})

What if we have multiple inputs to be taken in prompt
--> then there is a change in with_message_history
    we need to define a correct key to use to save chat to history

In [232]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according in {language}"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [233]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")],"language":"Hindi"})

AIMessage(content='नमस्ते Sachin Goyal! 😊  मैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे क्या पूछना चाहेंगे? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 30, 'total_tokens': 67, 'completion_time': 0.067272727, 'prompt_time': 0.002072055, 'queue_time': 0.246827655, 'total_time': 0.069344782}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d1315437-a7bd-4180-bec1-77be55ea44d9-0', usage_metadata={'input_tokens': 30, 'output_tokens': 37, 'total_tokens': 67})

In [234]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

whenever we reuse the older session for ex chat1 then it loads the old history and as we used english by default so now declaring any other language may not work for that and will print in english

In [239]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='Your name is Sachin Goyal.  \n\nIs there anything else I can help you with today? 🤔  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 996, 'total_tokens': 1021, 'completion_time': 0.045454545, 'prompt_time': 0.033659675, 'queue_time': 0.251460365, 'total_time': 0.07911422}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c74f3553-a440-486d-9b95-284757d442bd-0', usage_metadata={'input_tokens': 996, 'output_tokens': 25, 'total_tokens': 1021})

In [241]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Sachin Goyal")],"language":"Hindi"},config=config
)
response

AIMessage(content='नमस्ते, Sachin Goyal!  \n\nमुझे बहुत अच्छा लगा आपका नाम जानकर।  आपसे बातचीत करना मेरे लिए खुशी की बात है। 😊 \n\nक्या मैं आपकी कोई और मदद कर सकता हूँ?  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 149, 'total_tokens': 207, 'completion_time': 0.105454545, 'prompt_time': 0.006966222, 'queue_time': 0.24923800799999998, 'total_time': 0.112420767}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--497a6b01-4c5e-44c3-afd8-e6df2a6f881c-0', usage_metadata={'input_tokens': 149, 'output_tokens': 58, 'total_tokens': 207})

In [242]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='तुम्हारा नाम Sachin Goyal है! \n\nक्या मैं तुम्हें और कुछ बता सकता हूँ? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 221, 'total_tokens': 251, 'completion_time': 0.054545455, 'prompt_time': 0.010196478, 'queue_time': 0.247837231, 'total_time': 0.064741933}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec27a406-57f7-40d1-acad-4d7eac9f311e-0', usage_metadata={'input_tokens': 221, 'output_tokens': 30, 'total_tokens': 251})